In [2]:
import numpy as np; import random; import fractions; import math
from matplotlib import pyplot, patches
from matplotlib.patches import Ellipse
from sklearn.mixture import GaussianMixture 
from sklearn.metrics.cluster import silhouette_score
N_FEATURES=2; MIN_CENTERS=2; MAX_CENTERS=20

class Sample: 
    def __init__(self, n_samples, n_centers=3): 
        self.n_samples = n_samples
        self.n_centers = n_centers
    def _assign_positions(self, positions):
        self.positions = positions
        return
    def _assign_labels(self, labels):
        self.labels = labels
        return
    def reseed(self, new): 
        self.n_centers=new
        return
    def create_samples(self):
        pass   
class Normal_Blobs_Full(Sample): 
    def _semi_symm(self, n_dim_sqmat):  
        rng = np.random.default_rng()
        raw_mat = rng.uniform(low=0.0, high=0.1, size=(n_dim_sqmat, n_dim_sqmat))
        symm_mat = (raw_mat + np.transpose(raw_mat))/2 
        symm_semi_def_mat = np.dot(symm_mat, np.transpose(symm_mat))
        return symm_semi_def_mat
    def _assign_set_covariaces(self, set_cvmat): 
        self.set_cvmat = set_cvmat
        return 
    def _assign_set_means(self, set_means): 
        self.set_means = set_means
        return
    def assign_set_obliques(self, set_thetas): 
        self.set_thetas = set_thetas 
        return 
    def create_samples(self, n_feat=N_FEATURES):  
        rng = np.random.default_rng(); _frame=[0, 1]
        
        labels = np.ndarray(shape=(self.n_samples, 1), dtype=int)   
        set_lbl = np.linspace(start=0, stop=self.n_centers-1, num=self.n_centers-0, dtype=int) 
        for i in range(0, self.n_samples):  
            labels [i, 0] = rng.choice(set_lbl) 
        
        set_avg = rng.uniform(low=_frame[0], high=_frame[1], size=(self.n_centers, n_feat)) 
        set_cvmat = np.ndarray(shape=(self.n_centers, n_feat, n_feat))   
        for j in range(0, self.n_centers): 
            cov_ = self._semi_symm(n_feat)
            set_cvmat [j,:,:] = cov_
        
        positions = np.ndarray(shape=(self.n_samples, n_feat), dtype=float)
        for k in range(0, self.n_samples): 
            c_ = labels [k, 0]
            cv_ = rng.multivariate_normal(mean=set_avg [c_, :], cov=set_cvmat [c_, :, :]) 
            positions[k, :] = cv_
        self._assign_positions (positions); self._assign_labels (labels)
        return  
class Normal_Blobs_Identity(Sample): 
    def _assign_set_covariaces (self, set_cvmat): 
        self.set_cvmat = set_cvmat
        return 
    def _assign_set_means (self, set_means): 
        self.set_means = set_means
        return
    def assign_set_obliques (self, set_thetas): 
        self.set_thetas = set_thetas 
        return 
    def create_samples(self, n_feat=N_FEATURES): 
        VARIANCE =0.01
        rng = np.random.default_rng(); _frame=[0, 1] 
        labels = np.ndarray (shape=(self.n_samples, 1), dtype=int)   
        set_lbl = np.linspace (start=0, stop=self.n_centers-1, num=self.n_centers-0, dtype=int) 
        for i in range (0, self.n_samples):  
            labels [i, 0] = rng.choice (set_lbl) 
        
        set_avg = rng.uniform (low=_frame[0], high=_frame[1], size=(self.n_centers, n_feat))  
        set_cvmat = np.ndarray(shape=(self.n_centers, n_feat, n_feat))   
        for j in range(0, self.n_centers): 
            set_cvmat [j,:,:] = np.eye(N=n_feat, dtype=float) * VARIANCE
        
        positions = np.ndarray(shape=(self.n_samples, n_feat), dtype=float)
        for k in range(0, self.n_samples): 
            c_ = labels [k, 0]
            cv_ = rng.multivariate_normal(mean=set_avg [c_, :], cov=set_cvmat [c_, :, :]) 
            positions[k, :] = cv_
        self._assign_positions (positions); self._assign_labels (labels)
        return 
class Normal_Blobs_Sizes(Sample): 
    def _assign_set_covariaces (self, set_cvmat): 
        self.set_cvmat = set_cvmat
        return 
    def _assign_set_means (self, set_means): 
        self.set_means = set_means
        return
    def assign_set_obliques (self, set_thetas): 
        self.set_thetas = set_thetas 
        return 
    def create_samples(self, n_feat=N_FEATURES): 
        rng = np.random.default_rng(); _frame=[0, 1] 
        
        MIN_CLUST_VAR = 0.0001; MAX_CLUST_VAR = 0.01
        cluster_sizes = np.random.uniform (low=MIN_CLUST_VAR, high=MAX_CLUST_VAR, size=(self.n_centers, 1))
        
        labels = np.ndarray (shape=(self.n_samples, 1), dtype=int)   
        set_lbl = np.linspace (start=0, stop=self.n_centers-1, num=self.n_centers-0, dtype=int) 
        for i in range (0, self.n_samples):  
            labels [i, 0] = rng.choice (set_lbl) 
        
        set_avg = rng.uniform (low=_frame[0], high=_frame[1], size=(self.n_centers, n_feat))  
        set_cvmat = np.ndarray(shape=(self.n_centers, n_feat, n_feat))   
        for j in range(0, self.n_centers): 
            set_cvmat [j,:,:] = np.eye(N=n_feat, dtype=float) * cluster_sizes [j, 0]
        
        positions = np.ndarray(shape=(self.n_samples, n_feat), dtype=float)
        for k in range(0, self.n_samples): 
            c_ = labels [k, 0]
            cv_ = rng.multivariate_normal(mean=set_avg [c_, :], cov=set_cvmat [c_, :, :]) 
            positions[k, :] = cv_
        self._assign_positions (positions); self._assign_labels (labels)
        return 
class Normal_Blobs_Sparsity(Sample): 
    def _assign_set_covariaces (self, set_cvmat): 
        self.set_cvmat = set_cvmat
        return 
    def _assign_set_means (self, set_means): 
        self.set_means = set_means
        return
    def assign_set_obliques (self, set_thetas): 
        self.set_thetas = set_thetas 
        return 
    def create_samples(self, n_feat=N_FEATURES): 
        CLUSTER_VARIANCE = 0.001
        rng = np.random.default_rng(); _frame=[0, 1] 
        to_spar = np.random.uniform(low=0, high=1, size=(N_CENTERS, 1)) 
        sparsity = to_spar / np.sum(to_spar)
        a_lbl = np.linspace(start=0, stop=-1+self.n_centers, num=self.n_centers, dtype=int) 
        labels = rng.choice(size=(self.n_samples, 1), p=sparsity.T[0, :], a= a_lbl) 
        
        set_avg = rng.uniform (low=_frame[0], high=_frame[1], size=(self.n_centers, n_feat))  
        set_cvmat = np.ndarray(shape=(self.n_centers, n_feat, n_feat))   
        for j in range(0, self.n_centers): 
            set_cvmat [j,:,:] = np.eye(N=n_feat, dtype=float) * CLUSTER_VARIANCE
        
        positions = np.ndarray(shape=(self.n_samples, n_feat), dtype=float)
        for k in range(0, self.n_samples): 
            c_ = labels [k, 0]
            cv_ = rng.multivariate_normal(mean=set_avg [c_, :], cov=set_cvmat [c_, :, :]) 
            positions[k, :] = cv_
        self._assign_positions (positions); self._assign_labels (labels)
        return  
def _plot_log_likelihood(X, MIN_K, MAX_K, axsL, axsR, ll_=[], Ys=[], models=[]): 
    for it_ in range(MIN_K, MAX_K+1):
        estimator=GaussianMixture(n_components=it_, covariance_type='full')
        models.append(estimator)
        labels=estimator.fit_predict(X=X)
        ll_.append(estimator.score(X=X))
        Ys.append(labels)
    topScore=np.argmax(ll_)
    bestY,bestK,bestMod=Ys[topScore],topScore+(MIN_K),models[topScore]
    bestCov,bestMean=bestMod.covariances_,bestMod.means_
    
    S_=RGB_Set(n_clust=bestK) 
    S_._RGB_from_K()
    Ctensor=S_.Ctensor_
    
    axsR.scatter(x=X[:,0],y=X[:,1], c=colors_from_Y(Y=bestY, RGB_in=Ctensor))
    pocketsX = sortX(X=X, Y=bestY)
    for it_ in range(0, bestK):
        mean=bestMod.means_[it_,:]
        axsR.annotate(xy=tuple(mean.flatten()), text=r"$W[$" + str(it_)+ r"$]=$" 
                      + "%.2f" % _intracluster_distance(X=pocketsX[it_], mean=mean, sum_=float(0)))
        _add_ellipse(mean=mean, 
                     cov=bestMod.covariances_[it_,:,:], 
                     axs=axsR, facecolor=Ctensor[it_,:])
        
    axsL.scatter(y=ll_,x=np.linspace(start=MIN_K, stop=MAX_K, num=MAX_K-MIN_K+1))  
    axsL.set_ylabel('Log Likelihood'); axsL.set_xlabel('#clusters')
    return
def _plot_BIC(X, MIN_K, MAX_K, axsL, axsR, bic=[], Ys=[], models=[]):
    for it_ in range(MIN_K, MAX_K+1):
        estimator=GaussianMixture(n_components=it_, covariance_type='full')
        models.append(estimator)
        labels=estimator.fit_predict(X=X)
        bic.append(estimator.bic(X=X))
        Ys.append(labels)
    topScore=np.argmin(bic)
    bestY,bestK,bestMod=Ys[topScore],topScore+(MIN_K),models[topScore]
    bestCov,bestMean=bestMod.covariances_,bestMod.means_
    
    S_=RGB_Set(n_clust=bestK) 
    S_._RGB_from_K()
    Ctensor=S_.Ctensor_
    
    axsR.scatter(x=X[:,0],y=X[:,1], c=colors_from_Y(Y=bestY, RGB_in=Ctensor))
    pocketsX = sortX(X=X, Y=bestY)
    for it_ in range(0, bestK):
        mean=bestMod.means_[it_,:]
        axsR.annotate(xy=tuple(mean.flatten()), text=r"$W[$" + str(it_)+ r"$]=$" 
                      + "%.2f" % _intracluster_distance(X=pocketsX[it_], mean=mean, sum_=float(0)))
        _add_ellipse(mean=mean, 
                     cov=bestMod.covariances_[it_,:,:], 
                     axs=axsR, facecolor=Ctensor[it_,:])
        
    axsL.scatter(y=bic,x=np.linspace(start=MIN_K, stop=MAX_K, num=MAX_K-MIN_K+1))  
    axsL.set_ylabel('BIC'); axsL.set_xlabel('#clusters')
    return
def _plot_AIC(X, MIN_K, MAX_K, axsL, axsR, aic=[], Ys=[], models=[]):
    for it_ in range(MIN_K, MAX_K+1):
        estimator=GaussianMixture(n_components=it_, covariance_type='full')
        models.append(estimator)
        labels=estimator.fit_predict(X=X)
        aic.append(estimator.aic(X=X))
        Ys.append(labels)
    topScore=np.argmin(aic)
    bestY,bestK,bestMod=Ys[topScore],topScore+(MIN_K),models[topScore]
    bestCov,bestMean=bestMod.covariances_,bestMod.means_
    
    S_=RGB_Set(n_clust=bestK) 
    S_._RGB_from_K()
    Ctensor=S_.Ctensor_
    
    axsR.scatter(x=X[:,0],y=X[:,1], c=colors_from_Y(Y=bestY, RGB_in=Ctensor))
    pocketsX = sortX(X=X, Y=bestY)
    for it_ in range(0, bestK):
        mean=bestMod.means_[it_,:]
        axsR.annotate(xy=tuple(mean.flatten()), text=r"$W[$" + str(it_)+ r"$]=$" 
                      + "%.2f" % _intracluster_distance(X=pocketsX[it_], mean=mean, sum_=float(0)))
        _add_ellipse(mean=mean, 
                     cov=bestMod.covariances_[it_,:,:], 
                     axs=axsR, facecolor=Ctensor[it_,:])
        
    axsL.scatter(y=aic,x=np.linspace(start=MIN_K, stop=MAX_K, num=MAX_K-MIN_K+1))  
    axsL.set_ylabel('AIC'); axsL.set_xlabel('#clusters')
    return
def _plot_silhouette(X, MIN_K, MAX_K, axsL, axsR, sihouette=[], Ys=[], models=[]):
    for it_ in range(MIN_K, MAX_K+1):
        estimator=GaussianMixture(n_components=it_, covariance_type='full')
        models.append(estimator)
        labels=estimator.fit_predict(X=X)
        sihouette.append(silhouette_score(X=X, labels=labels))
        Ys.append(labels)
    topScore=np.argmax(sihouette)
    bestY,bestK,bestMod=Ys[topScore],topScore+(MIN_K),models[topScore]
    bestCov,bestMean=bestMod.covariances_,bestMod.means_
    
    S_=RGB_Set(n_clust=bestK) 
    S_._RGB_from_K()
    Ctensor=S_.Ctensor_
    
    axsR.scatter(x=X[:,0],y=X[:,1], c=colors_from_Y(Y=bestY, RGB_in=Ctensor))
    pocketsX = sortX(X=X, Y=bestY)
    for it_ in range(0, bestK):
        mean=bestMod.means_[it_,:]
        axsR.annotate(xy=tuple(mean.flatten()), text=r"$W[$" + str(it_)+ r"$]=$" 
                      + "%.2f" % _intracluster_distance(X=pocketsX[it_], mean=mean, sum_=float(0)))
        _add_ellipse(mean=mean, 
                     cov=bestMod.covariances_[it_,:,:], 
                     axs=axsR, facecolor=Ctensor[it_,:])
    
    axsL.scatter(y=sihouette,x=np.linspace(start=MIN_K, stop=MAX_K, num=MAX_K-MIN_K+1))  
    axsL.set_ylabel('silhouette'); axsL.set_xlabel('#clusters')
    return
def sortX(X, Y): 
    assert(X.shape[0]==Y.shape[0])
    lenY=Y.shape[0]; n_mix=np.unique(Y).size 
    pockets=[None] * n_mix
    for iter_0 in range(0, n_mix):
        stackX = X[0:1,:]
        for iter_1 in range(0, lenY):
            tag=Y[iter_1]
            if iter_0 == tag:
                apX=X[iter_1:iter_1+1,:] 
                stackX = np.concatenate((stackX, apX), axis=0)
        pockets[iter_0]=stackX[1:,]
    return pockets
def _intracluster_distance(X, mean, sum_=float(0)):
    for row in X: 
        sum_ += np.linalg.norm(row.T - mean.flatten())
    return sum_/X.shape[0] 
def _add_ellipse(axs, mean, cov, facecolor, SCALE=5.99):
    eVal, eVec=np.linalg.eig(a=cov)  
    ratio=eVec[1,0]/eVec[1,1]
    scaled_eVal=np.sqrt(eVal*SCALE)*2
    elp=Ellipse(xy=tuple(mean.flatten()), angle=np.arctan(ratio)* 180 / np.pi,
                width=scaled_eVal[0], height=scaled_eVal[1],fc=facecolor)
    axs.add_patch(elp).set_alpha(0.40)
    return
def colors_from_Y(Y, RGB_in, DIM_RGB=3): 
    colors = np.ndarray(shape=(Y.shape[0], DIM_RGB), dtype=float)
    for it_ in range(0, Y.shape[0]): 
        tag_=Y[it_]
        colors[it_,:] = RGB_in[tag_,:]
    return colors
class RGB_Set: 
    def __init__(self, n_clust):
        self.n_clust=n_clust 
        pass
    def _RGB_from_K(self,DIM_RBG=3): 
        rng=np.random.default_rng()
        self.Ctensor_=rng.integers(low=0,high=256,size=(self.n_clust, DIM_RBG)) /256
        return 
def plot_metrics(X, MIN_K=2, MAX_K=10): 
    fig, axs=pyplot.subplots(nrows=2,ncols=3) 
    fig.patch.set_facecolor ('#E0E0E0') 
    pyplot.subplots_adjust(right=1.5, wspace=.50, hspace=0.3, top=1)
    _plot_silhouette(X=X,MIN_K=MIN_K,MAX_K=MAX_K,axsL=axs[0,0],axsR=axs[1,0])
    _plot_AIC(X=X,MIN_K=MIN_K,MAX_K=MAX_K,axsL=axs[0,1], axsR=axs[1,1])
    _plot_BIC(X=X,MIN_K=MIN_K,MAX_K=MAX_K,axsL=axs[0,2], axsR=axs[1,2])
    pyplot.show(); pyplot.close()
    return 

In [96]:
def _empirical_mean(X:np.ndarray): 
    return np.sum(X)/X.size 
class Visual():  
    def createSamCopies(self, batchsize, n_copies):  
        (blobs:=Normal_Blobs_Full(n_samples=batchsize)).create_samples() 
        self.X=np.zeros(shape=(n_copies,blobs.positions.shape[0],blobs.positions.shape[1]))
        for i in range(0,n_copies):  
            self.X[i,:,:]= blobs.positions
        return
    def run(self, range_mix:tuple, measurement='aic'):
        #init metric schemes
        self.range_mix=range_mix
        #GMM copies x blobs  
        n_mix=1+range_mix[1]-range_mix[0]
        n_copies=self.X.shape[0]
        metrics=np.ndarray(shape=(n_mix,n_copies))
        for mix_ in range(0,n_mix): 
            for i in range(0,n_copies):
                self.mixer=GaussianMixture(n_components=mix_+range_mix[0],random_state=0)
                labels=self.mixer.fit_predict(X=self.X[i,:,:])
                metrics[mix_,i]=self._calculate_metric(X=self.X[i,:,:],labels=labels,measurement=measurement)

        # empiracal mean silhouettes
        self.mean_silhouttes=np.ndarray(shape=(1,n_mix))
        for i in range(0, n_mix):
            self.mean_silhouttes[0,i]=_empirical_mean(X=metrics[i,:])
        return 
    def _calculate_metric(self,X,labels,measurement:str): 
        if measurement=='silhouette': metric_=silhouette_score(X=X,labels=labels)
        elif measurement=='aic': metric_= self.mixer.aic(X) 
        elif measurement=='bic': metric_=self.mixer.bic(X)
        return metric_
(charter:=Visual()).createSamCopies(batchsize=100,n_copies=10) 
charter.run(range_mix=(3,4))